In [38]:
import tensorflow as tf
import tensorflow_gan as tfgan
import numpy as np
from numpy import zeros
from numpy import ones
from numpy import hstack
from numpy.random import rand
from numpy.random import randn
from numpy.random import random
from numpy.random import choice
from numpy.random import randint
import pandas as pd
import glob
import os
from os import makedirs
import PIL
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU
from keras.utils.vis_utils import plot_model
import seaborn as sb
from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pydot
from keras.initializers import RandomNormal
from keras.optimizers import Adam
sb.set()

In [2]:
import os 
import earthpy as et 

In [3]:
## Code needed to set a new working directory

#my_path = os.path.join(et.io.HOME, 'recommender_system')
#os.mkdir(my_path)

## Set Working Directory
os.chdir(os.path.join(et.io.HOME, 'recommender_system'))

In [4]:
# Check if it worked
os.getcwd()

'/Users/chriskamerin/recommender_system'

In [5]:
df =pd.read_csv('data_in_use/df2.csv')

In [6]:
df = df.sample(frac=1).reset_index(drop=True)

In [7]:
df['user_id']  = pd.Categorical(df.user_id).codes
df['headphone_name']  = pd.Categorical(df.headphone_name).codes

In [8]:
#Set columns we want to normalize

cols_to_norm = ['sound_quality_importance', 'positional_importance', 'loud_importance',
       'mic_importance', 'wireless_importance','noise_canceling_importance', 'in_ear_importance',
       'on_ear_importance', 'over_ear_importance', 'no_pref_oninover', 'recording_quality', 'noise_handling','user_rating',
       'Impedance', 'sensitivity(dbv)', 'accuracy',
       'weighted_harmonic_distortion', 'soundstage', 'imaging',
       'noise_isolation', 'wireless']
imp_col = ['sound_quality_importance', 'positional_importance', 'loud_importance',
       'mic_importance', 'wireless_importance', 
#       'price_sensitivity'
]
#price_col = ['max_price','price']
hp_feat_col = ['recording_quality', 'noise_handling', 'accuracy',
       'weighted_harmonic_distortion', 'soundstage', 'imaging',
       'noise_isolation', 'wireless']
remaining_col = ['Impedance', 'sensitivity(dbv)']



In [9]:
df =df[['game_use', 'rec_use', 'studio_use', 'exercise_use',
       'office_use', 'podcast_use', 'travel_use', 'other_use',
       'sound_quality_importance', 'positional_importance', 'loud_importance',
       'mic_importance', 'wireless_importance', 'in_ear_importance',
       'on_ear_importance', 'over_ear_importance', 'no_pref_oninover','attached_mic', 'recording_quality',
       'noise_handling', 'Impedance', 'sensitivity(dbv)', 'accuracy',
      'weighted_harmonic_distortion', 'soundstage', 'imaging',
      'noise_isolation', 'wireless', 'over_ear_style', 'on_ear_style',
      'in_ear_style', 'open_back', 'close_back', 'semiclosed_back']]
#ydf =df[[ 'attached_mic', 'recording_quality',
#       'noise_handling', 'Impedance', 'sensitivity(dbv)', 'accuracy',
#      'weighted_harmonic_distortion', 'soundstage', 'imaging',
#      'noise_isolation', 'wireless', 'over_ear_style', 'on_ear_style',
#       'in_ear_style', 'open_back', 'close_back', 'semiclosed_back']]
#xtest=df1[['game_use', 'rec_use', 'studio_use', 'exercise_use',
#       'office_use', 'podcast_use', 'travel_use', 'other_use',
#       'sound_quality_importance', 'positional_importance', 'loud_importance',
#       'mic_importance', 'wireless_importance', 'in_ear_importance',
#       'on_ear_importance', 'over_ear_importance', 'no_pref_oninover']][12:]
#ytest=df1[[ 'attached_mic', 'recording_quality',
#       'noise_handling', 'Impedance', 'sensitivity(dbv)', 'accuracy',
#       'weighted_harmonic_distortion', 'soundstage', 'imaging',
#       'noise_isolation', 'wireless', 'over_ear_style', 'on_ear_style',
#       'in_ear_style', 'open_back', 'close_back', 'semiclosed_back']][12:]

In [10]:
#normalize columns in df

df=df.fillna(0)
df[imp_col] = df[imp_col].apply(lambda x: (x - 0) / (10 - 0))
#df[price_col] = df[price_col].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
df[hp_feat_col] = df[hp_feat_col].apply(lambda x: (x - x.min()) / (x.max() - x.min()))   
df[remaining_col] = df[remaining_col].apply(lambda x: (x - x.min()) / (x.max() - x.min()))       
#df[cols_to_norm] = df[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

In [52]:
# define the standalone discriminator model
def define_discriminator(n_inputs=34):
	model = Sequential()
	init = RandomNormal(mean=0.0, stddev=0.02)
	model.add(Dense(256, kernel_initializer=init, input_dim=n_inputs))
	model.add(LeakyReLU(alpha=0.1))
	model.add(Dense(256, kernel_initializer=init))
	model.add(LeakyReLU(alpha=0.1))
	model.add(Dense(256, kernel_initializer=init))
	model.add(LeakyReLU(alpha=0.1))
	model.add(Dense(34, activation='sigmoid'))
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model
 
# define the standalone generator model
def define_generator(latent_dim, n_outputs=34):
	model = Sequential()
	init = RandomNormal(mean=0.0, stddev=0.02)
	model.add(Dense(256, kernel_initializer=init, input_dim=latent_dim))
	model.add(LeakyReLU(alpha=0.1))
	model.add(Dense(256, kernel_initializer=init))
	model.add(LeakyReLU(alpha=0.1))
	model.add(Dense(256, kernel_initializer=init))
	model.add(LeakyReLU(alpha=0.1))
	model.add(Dense(n_outputs, activation='linear'))
	return model
 
# define the combined generator and discriminator model, for updating the generator
def define_gan(generator, discriminator):
	# make weights in the discriminator not trainable
	discriminator.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(generator)
	# add the discriminator
	model.add(discriminator)
	# compile model
	model.compile(loss='binary_crossentropy', optimizer='adam')
	return model

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n):
	# generate points in the latent space
	x_input = randn(latent_dim * n)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n, latent_dim)

	#Dimension Diagnostic
	#x_in_lat.append(x_input)

	return x_input

# randomly flip some labels
def noisy_labels(y, p_flip):
	# determine the number of labels to flip
	n_select = int(p_flip * y.shape[0])
	# choose labels to flip
	flip_ix = choice([i for i in range(y.shape[0])], size=n_select)
	# invert the labels in place
	y[flip_ix] = 1 - y[flip_ix]
	return y


def smooth_positive_labels(y):
	return y - 0.3 + (random(y.shape) * 0.5)

#Dimension Diagnostic
#real_y = [] 
# generate n real samples with class labels
def generate_real_samples(n):
	# generate class labels
	y = ones((1, n))

	y = smooth_positive_labels(y)
	#Dimension Diagnostic
	#real_y.append(y)

	return  y

# select real samples
#def generate_real_samples(dataset, n_samples):
	# choose random instances
#	ix = randint(0, df.shape[0], n_samples)
	# select rows
#	X = df[ix:]
	# generate class labels
#	y = ones((1, n))
#	y = smooth_positive_labels(y)
#	return X, y

#Dimension Diagnostic
#latent_x_input= []


def smooth_negative_labels(y):
	return y + random(y.shape) * 0.3

#Dimension Diagnostic 
#gen_X_fake =[]
#gen_y_fake =[]
#gen_x_input=[]

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n):
	# generate points in latent space
	x_input = generate_latent_points(latent_dim, n)

	#Dimension Diagnostic
	#x_in_gen.append(x_input)

	# predict outputs
	X = generator.predict(x_input)
	# create class labels
	y = zeros((n, 34))
	y = smooth_negative_labels(y)

	#Dimension Diagnostic
	#y_fake.append(y)
	#X_fake.append(X)

	return X, y
	
	
# evaluate the discriminator and plot real and fake points
def summarize_performance(epoch, generator, discriminator, latent_dim, n=34):
	# prepare real samples
	for row in range(len(df)):
		x_real, y_real = df[row:row+1], generate_real_samples(n)

	#x_real, y_real = generate_real_samples(df, 34)

	# evaluate discriminator on real examples
	_, acc_real = discriminator.evaluate(x_real, y_real, verbose=0)
	# prepare fake examples
	x_fake, y_fake = generate_fake_samples(generator, latent_dim, 1)
	# evaluate discriminator on fake examples
	_, acc_fake = discriminator.evaluate(x_fake, y_fake, verbose=0)
	# summarize discriminator performance
	print(epoch, acc_real, acc_fake)
	# scatter plot real and fake data points
	#pyplot.scatter(x_real[:, 0], x_real[:, 1], color='red')
	#pyplot.scatter(x_fake[:, 0], x_fake[:, 1], color='blue')
	#pyplot.show()
 
#Dimension Diagnostic
model_x_fake = []
#model_y_fake = []
#model_x_real = []
#model_y_real = []
#model_x_gan=[]
#model_y_gan=[]


# create a line plot of loss for the gan and save to file
def plot_history(d1_hist, d2_hist, g_hist, a1_hist, a2_hist):
	# plot loss
	pyplot.subplot(2, 1, 1)
	pyplot.plot(d1_hist, label='d-real')
	pyplot.plot(d2_hist, label='d-fake')
	pyplot.plot(g_hist, label='gen')
	pyplot.legend()
	# plot discriminator accuracy
	pyplot.subplot(2, 1, 2)
	pyplot.plot(a1_hist, label='acc-real')
	pyplot.plot(a2_hist, label='acc-fake')
	pyplot.legend()
	# save plot to file
	pyplot.savefig('results_baseline/plot_line_plot_loss.png')
	pyplot.close()

# train the generator and discriminator
def train(g_model, d_model, gan_model, latent_dim, n_epochs=100, n_batch=68, n_eval=20):
	
# prepare lists for storing stats each iteration
	d1_hist, d2_hist, g_hist, a1_hist, a2_hist = list(), list(), list(), list(), list()
# manually enumerate epochs
	for i in range(n_epochs):
		# prepare real samples
		#for row in range(len(df)):
			#x_real, y_real = df[row:row+1], generate_real_samples(n)

		x_real, y_real = df[i%len(df):i%len(df)+1], generate_real_samples(34)

		#x_real, y_real = generate_real_samples(df, 34)

		#Dimension Diagnostic
		#model_x_real.append(x_real)
		#model_y_real.append(y_real)

		# prepare fake examples
		x_fake, y_fake = generate_fake_samples(g_model, latent_dim, 1)

		#Dimension Diagnostic
		model_x_fake.append(x_fake)
		#model_y_fake.append(y_fake)

		# update discriminator
		y_real= noisy_labels(y_real,.05)
		y_fake= noisy_labels(y_fake,.05)
		d_loss1, d_acc1 = d_model.train_on_batch(x_real, y_real)
		d_loss2, d_acc2 = d_model.train_on_batch(x_fake, y_fake)
		# prepare points in latent space as input for the generator
		x_gan = generate_latent_points(latent_dim, 1)
		# create inverted labels for the fake samples
		y_gan = ones((1, 34))

		#Dimension Diagnostic
		#model_x_gan.append(x_gan)
		#model_y_gan.append(y_gan)

		# update the generator via the discriminator's error
		g_loss = gan_model.train_on_batch(x_gan, y_gan)
		# evaluate the model every n_eval epochs
	
		# summarize loss on this batch
		#print('>%d, d1=%.3f, d2=%.3f g=%.3f, a1=%d, a2=%d' %
		#	(i+1, d_loss1, d_loss2, g_loss, int(100*d_acc1), int(100*d_acc2)))
		# record history
		d1_hist.append(d_loss1)
		d2_hist.append(d_loss2)
		g_hist.append(g_loss)
		a1_hist.append(d_acc1)
		a2_hist.append(d_acc2)
		# evaluate the model performance every 'epoch'
		if (i+1) % n_eval == 0:
			summarize_performance(i, g_model, d_model, latent_dim)

	plot_history(d1_hist, d2_hist, g_hist, a1_hist, a2_hist)


# make folder for results
makedirs('results_baseline', exist_ok=True)
# size of the latent space
latent_dim = 100
# create the discriminator
discriminator = define_discriminator()
# create the generator
generator = define_generator(latent_dim)
# create the gan
gan_model = define_gan(generator, discriminator)
# train model
train(generator, discriminator, gan_model, latent_dim)

19 0.0 0.0
39 0.0 0.0
59 0.0 0.0
79 0.0 0.0
99 1.0 0.0


In [12]:
#Dimension Checks

In [13]:
#len(real_y),real_y[1].shape,real_y[2].shape,real_y[2].shape

In [14]:
#len(model_x_real),model_x_real[1].shape,model_x_real[2].shape

In [15]:
#len(model_y_real),model_y_real[1].shape,model_y_real[2].shape

In [16]:
#len(model_x_fake), model_x_fake{1].shape,model_x_fake[2].shape

In [17]:
#len(model_y_fake), model_y_fake[1].shape,model_y_fake[2].shape

In [18]:
#len(model_x_gan),model_x_gan[1].shape,model_x_gan2].shape

In [19]:
#len(model_y_gan),model_y_gan[1].shape,model_y_gan[2].shape

In [20]:
#len(latent_x_input),latent_x_input[1].shape,latent_x_input[2].shape,latent_x_input[3].shape,latent_x_input[4].shape,

In [21]:
#len(gen_x_input),gen_x_input[1].shape,gen_x_input[2].shape,gen_x_input[3].shape,

In [22]:
#len(gen_X_fake), gen_X_fake[1].shape,gen_X_fake[2].shape,gen_X_fake{3].shape,gen_X_fake[4].shape

In [23]:
#len(gen_y_fake), gen_y_fake[1].shape,gen_y_fake[2].shape,gen_y_fake[3].shape,gen_y_fake[4].shape

In [50]:
model_x_fake


574,  0.13153492, -0.05732365,
          0.0393037 ,  0.04404167,  0.07069601,  0.01945558]],
       dtype=float32),
 array([[ 0.06822327,  0.11430866,  0.0543235 ,  0.16902661,  0.12597257,
          0.14293984,  0.06857334, -0.01123631,  0.11747857,  0.04223321,
         -0.0572692 ,  0.0589719 , -0.05313965,  0.1681132 , -0.00268531,
          0.04084631,  0.00709485,  0.14570841, -0.02301747, -0.0468325 ,
         -0.03087743,  0.06324296,  0.00921736, -0.07036915,  0.21995015,
         -0.01058761, -0.02013329, -0.05223878,  0.17574334, -0.08519697,
          0.02872556,  0.06674926,  0.08705831,  0.04953057]],
       dtype=float32),
 array([[ 0.18559094,  0.21693203,  0.14160477,  0.18699443,  0.20387095,
          0.23904946,  0.07830602,  0.01073819,  0.20560263,  0.11470419,
         -0.0828713 ,  0.08088544, -0.05790745,  0.19028321,  0.01171534,
          0.09494632,  0.12284372,  0.22793601, -0.00852719,  0.00542408,
          0.04196319,  0.16703366, -0.00154871,  0.057733

In [25]:
x_fake

NameError: name 'x_fake' is not defined